Problem statement :

A QA chatbot (interacting with a pdf or word document ) is shown.

The first method to come up with a QA chatbot is to use a load_QA_chain, with a chain type : stuff (taking entire document) or with map_reduce (separates document to batches if the document is long, getting a summary). 


Since the allowed number of context-length tokens is 4097, the chain type could be changed to map-reduce to address this problem.


https://github.com/hwchase17/langchain/issues/1349


In [1]:
!pip install langchain openai chromadb tiktoken pypdf panel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.7/993.7 kB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.3/249.3 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.6/938.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import langchain
import openai
import os
import chromadb
import tiktoken
import pypdf
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import panel as pn

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "sk...7d"
openai.api_key = "sk...7d"

#document loader

In [4]:
loader = PyPDFLoader("C06.pdf")
documents = loader.load()

# QA function

Chain type, embeddings, similarity search. 


In [5]:
def qa(file, query, chain_type, k):
  loader = PyPDFLoader(file)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size= 1000, chunk_overlap = 0)
  texts = text_splitter.split_documents(documents)
  embeddings = OpenAIEmbeddings()
  db = Chroma.from_documents(texts, embeddings)
  retriever = db.as_retriever(search_type = "similarity", search_kwargs= {"k": k})
  qa = RetrievalQA.from_chain_type(
      llm = OpenAI(), chain_type = chain_type, retriever = retriever, return_source_documents = True
  )
  result = qa({'query': query})
  print(result['result'])
  return result


In [6]:
result = qa("C06.pdf", "what are some objectives mentioned about AAP?", "map_reduce",2)

 The objective of AAP is to create and manage network animations to promote the sales of new vehicles.


In [7]:
#result